In [ ]:
import networkx as nx
import numpy as np

def generate_regular_graph():
    # 这里简单以正则图为例, 鼓励同学们尝试在其他类型的图(具体可查看如下的nx文档)上测试算法性能
    # nx文档 https://networkx.org/documentation/stable/reference/generators.html
    graph = nx.random_graphs.random_regular_graph(d=99, n=200, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

def generate_erdos_renyi_graph():
    graph = nx.random_graphs.erdos_renyi_graph(n=180, p=0.52, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

data_path = 'ER'

if data_path == 'regular':
    graph, n_nodes, n_edges = generate_regular_graph()
elif data_path == 'ER':
    graph, n_nodes, n_edges = generate_erdos_renyi_graph()
    
print(n_nodes, n_edges)
def get_fitness(graph, x, threshold=0):
    # 获得Cuts值需要将图分为两部分, 这里默认以0为阈值把解分成两块.
    g1 = np.where(x == 0)[0]
    g2 = np.where(x == 1)[0]
    return -nx.cut_size(graph, g1, g2) / n_edges

In [ ]:
from scipy.linalg import lstsq
import numpy as np
k = 8
n = n_nodes
population = [np.zeros((n))]
popSize = 1
fitness = [np.zeros(2)]
fitness[0][0] = np.inf
fitness[0][1] = population[0].sum()
T = round(n*2*np.exp(1))
epoch = T // 10

mse = []
r2 = []

tt = []
g_ = 0

for mm in range(k):
    for _ in range(T):
        g_ += 1
        if g_ and g_ % epoch == 0:
            m_fitness, m_population = np.array(fitness), np.array(population)
            temp = m_fitness[:,1]<=k
            j = max(m_fitness[temp,1])
            seq = m_fitness[:,1] == j
            selectedVariables = m_population[seq,:][0]
            loss = get_fitness(graph, selectedVariables)
            mse.append(loss)
            tt.append(g_)
            print(loss)
            
        idx = np.random.randint(popSize,size=1)[0]
        pm = np.random.choice((0,1), p=(1-1/n, 1/n), size=n)
        flip = 1 - population[idx]
        offspring = np.where(pm, flip, population[idx])
        offspringFit = np.zeros(2)
        offspringFit[1] = offspring.sum()
        if offspringFit[1] == 0 or offspringFit[1] >= 2*k:
            offspringFit[0] = np.inf
        else:
            offspringFit[0] = get_fitness(graph, offspring)
        
        m_fitness, m_population = np.array(fitness), np.array(population)
        if sum((m_fitness[:,0]<offspringFit[0])*(m_fitness[:,1]<=offspringFit[1]))+\
            sum((m_fitness[:,0]<=offspringFit[0])*(m_fitness[:,1]<offspringFit[1]))>0:
            continue
        else:
            deleteIndex = np.array((m_fitness[:,0]>=offspringFit[0])*(m_fitness[:,1]>=offspringFit[1]))
        ndelete = ~deleteIndex
        population = list(m_population[ndelete]) + [offspring]
        fitness = list(m_fitness[ndelete]) + [offspringFit]
        popSize = len(population)
        
    m_fitness, m_population = np.array(fitness), np.array(population)
    deleteIndex = np.array(m_fitness[:,1]<(mm+1))
    ndelete = ~deleteIndex
    population = list(m_population[ndelete])
    fitness = list(m_fitness[ndelete])
    popSize = len(population)
    
m_fitness, m_population = np.array(fitness), np.array(population)
temp = m_fitness[:,1]<=k
j = max(m_fitness[temp,1])
seq = m_fitness[:,1] == j
selectedVariables = m_population[seq,:]

In [ ]:
m_fitness, m_population = np.array(fitness), np.array(population)
temp = m_fitness[:,1]<=k
j = max(m_fitness[temp,1])
seq = m_fitness[:,1] == j
selectedVariables = m_population[seq,:]

In [ ]:
print(j)
print(m_fitness[m_fitness[:,1]<=k])

In [ ]:
for selectedVariable in selectedVariables:
    loss = get_fitness(graph, selectedVariable)
    print(loss)
# [[0.00788808]]
# [[0.99211192]]

In [ ]:
from matplotlib import pyplot as plt

mse = np.array(mse)
plt.plot(tt, mse)

In [ ]:
ep = m_fitness[m_fitness[:,1]<=2*k].T
np.save(f'DPOSS_{data_path}_t.npy', tt)
np.save(f'DPOSS_{data_path}_mse.npy', mse)
np.save(f'DPOSS_{data_path}_pareto.npy', ep)